# Classification de texte - Articles arabes

## Dépendences

In [1]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 3279159296)]

In [1]:
from keras.layers import Embedding, Dense, LSTM, Dropout, Input
#from keras.layers import GRU, MaxPooling1D, Conv1D, Flatten
from keras.preprocessing import text, sequence
from keras.models import Model
from keras.utils import np_utils

from gensim.models.keyedvectors import KeyedVectors

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from utils import *

Using TensorFlow backend.


## Préparation de données

In [2]:
# language model
WORD_MODEL, _, MAX_FEATURES, EMBED_SIZE = get_init_parameters('./arabic_bins/web_cbow_300')

# load data
data_paths = ['arabic_dataset_classifiction.csv']
x_train, x_test, y_train, y_test, MAX_TEXT_LENGTH = split_datasets(data_paths, test_size=0.1)
CLASSES_LIST = np.unique(y_train)
print('Label categories: ' + str(CLASSES_LIST))

# 
x_train, x_test, y_train, y_test, train_y_cat, word_index = class_str_2_ind(x_train, x_test, 
                                                                            y_train, y_test,
                                                                            CLASSES_LIST, MAX_FEATURES,
                                                                            MAX_TEXT_LENGTH)

Dataset splited.
Label categories: ['0' '1' '2' '3' '4']
Converting data to trainable form...
Number of training examples: 100555
Number of testing examples: 11173


## Apprentissage du modèle

In [3]:
model = get_main_model(word_index, WORD_MODEL, EMBED_SIZE, MAX_TEXT_LENGTH)
del WORD_MODEL
del word_index

Building embedding matrix...
Embedding matrix built.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5049)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5049, 300)         121404600 
_________________________________________________________________
lstm_1 (LSTM)                (None, 5049, 300)         721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5049, 300)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)        

In [6]:
x_train.shape

(100555, 5049)

In [4]:
history, score = train_fit_predict(model, x_train[:10000], x_test, train_y_cat[:10000], batch_size=20, epochs=1)

Epoch 1/1
10000/10000 [==============================] - 6157s 616ms/step - loss: 1.5497 - acc: 0.4180


In [9]:
score

array([[ 0.16101843,  0.17298171,  0.16452436,  0.19388433,  0.30759114],
       [ 0.16101843,  0.17298171,  0.16452436,  0.19388433,  0.30759114],
       [ 0.16101843,  0.17298171,  0.16452436,  0.19388433,  0.30759114],
       ..., 
       [ 0.16101843,  0.17298171,  0.16452436,  0.19388433,  0.30759114],
       [ 0.16101843,  0.17298171,  0.16452436,  0.19388433,  0.30759114],
       [ 0.16101843,  0.17298171,  0.16452436,  0.19388433,  0.30759114]], dtype=float32)